In [1]:
import pandas as pd
import os
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from gensim import matutils,corpora, models
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not 

In [2]:
negative_list = os.listdir("negative_reviews") # names of all files in the negative_polarity dir into a list
positive_list = os.listdir("positive_reviews") # names of all files in the positive_polarity dir into a list

In [3]:
def preprocess(files_list,root_dir,polarity):
    labeled_class = []
    reviews = []
    actual_class =[]
    for j in files_list:
        labeled_class.append(polarity)
        k = str(open( root_dir + '/' + j).read())
        reviews.append(k)
        actual_class.append(str(j.split('_')[0]))
    data = pd.DataFrame({'labeled_class':labeled_class,'review':reviews,'actual_class':actual_class})
    return data

In [4]:
negative_df = preprocess(negative_list,'negative_reviews','negative')
positive_df = preprocess(positive_list,'positive_reviews','positive')

In [5]:
negative_df.head()

,labeled_class,review,actual_class
0,negative,My wife and I just spent a long weekend at the...,t
1,negative,The historic feel of the hotel really had a st...,t
2,negative,I haven't actually stayed at this hotel- yet- ...,t
3,negative,I was very much looking forward to our stay at...,t
4,negative,The hotel is almost always very helpful. This ...,t


In [6]:
positive_df.head()

,labeled_class,review,actual_class
0,positive,The Hard Rock Hotel Chicago has become my favo...,t
1,positive,My wife and I had to stay downtown for event t...,t
2,positive,took a weekend trip with my wife. got a great ...,t
3,positive,We stayed here because of all the great review...,t
4,positive,Just returned from a week in Chicago with the ...,t


In [7]:
target = []
for i in positive_df.index:
    if ((positive_df['labeled_class'][i] == 'positive') & (positive_df['actual_class'][i] == 't')):
        target.append(2)
    elif ((positive_df['labeled_class'][i] == 'positive') & (positive_df['actual_class'][i] == 'd')):
        target.append(1)
    else:
        print('Error!')
positive_df['target'] = target

In [8]:
target = []
for i in negative_df.index:
    if ((negative_df['labeled_class'][i] == 'negative') & (negative_df['actual_class'][i] == 't')):
        target.append(3)
    elif ((negative_df['labeled_class'][i] == 'negative') & (negative_df['actual_class'][i] == 'd')):
        target.append(4)
    else:
        print('Error!')
negative_df['target'] = target

In [9]:
data = positive_df.merge(negative_df,how='outer')

In [10]:
data = data[['review','target']]

In [11]:
data.head()

,review,target
0,The Hard Rock Hotel Chicago has become my favo...,2
1,My wife and I had to stay downtown for event t...,2
2,took a weekend trip with my wife. got a great ...,2
3,We stayed here because of all the great review...,2
4,Just returned from a week in Chicago with the ...,2


In [12]:
data.target.value_counts()

4    410
3    400
2    400
1    400
Name: target, dtype: int64

In [13]:
#Using Spacy to get tokens and pos tags

In [14]:
nlp=spacy.load('en',entity='false');

In [15]:
def extract_tokens(df):
    review_tokenized = []
    lmt = WordNetLemmatizer()
    for index, datapoint in df.iterrows():
        tokenize_words = nlp.tokenizer(datapoint["review"])
       # pos_word = tokenize_words.pos_
        #tokenize_words = ["_".join([lmt.lemmatize(i[0]),i[1]]) 
         #               for i in pos_word if (i[0] not in stopwords.words("english") and len(i[0]) > 2)]
        review_tokenized.append(tokenize_words)
    df["review_tokenized"] = review_tokenized
    return df

data = extract_tokens(data)
data

,review,target,review_tokenized
0,The Hard Rock Hotel Chicago has become my favo...,2,"(The, Hard, Rock, Hotel, Chicago, has, become,..."
1,My wife and I had to stay downtown for event t...,2,"(My, wife, and, I, had, to, stay, downtown, fo..."
2,took a weekend trip with my wife. got a great ...,2,"(took, a, weekend, trip, with, my, wife, ., go..."
3,We stayed here because of all the great review...,2,"(We, stayed, here, because, of, all, the, grea..."
4,Just returned from a week in Chicago with the ...,2,"(Just, returned, from, a, week, in, Chicago, w..."
5,We stayed at the Swissotel for a special birth...,2,"(We, stayed, at, the, Swissotel, for, a, speci..."
6,I just left the Conrad Chicago and have nothin...,2,"(I, just, left, the, Conrad, Chicago, and, hav..."
7,"This is a great hotel! The views are fabulous,...",2,"(This, is, a, great, hotel, !, The, views, are..."
8,Got a good deal on a weekend pacage in this Ho...,2,"(Got, a, good, deal, on, a, weekend, pacage, i..."
9,"Thirty years ago, we had a tiny ""room"" and ind...",2,"(Thirty, years, ago, ,, we, had, a, tiny, "", r..."


In [16]:
#pos_tags=[];
#g=[[]];
#for datapoint in data['review_tokenized']:
 #   s=""
  #  for j in datapoint:
   #     s=s+str(j)+"_"+j.pos_;
    #    s=s+","
    #t=s;    
    #pos_tags.append(t)  
    #g.append(pos_tags)
#print(g) 


In [17]:
tokens = []
lemma = []
pos = []
for doc in nlp.pipe(data['review'].astype('unicode').values, batch_size=50,
                        n_threads=3):
    if doc.is_parsed:
        tokens.append(str([n.text for n in doc]))
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)   
        
data['species_tokens'] = tokens
data['species_lemma'] = lemma
data['species_pos'] = pos

In [18]:
data['species_tokens']

0       ['The', 'Hard', 'Rock', 'Hotel', 'Chicago', 'h...
1       ['My', 'wife', 'and', 'I', 'had', 'to', 'stay'...
2       ['took', 'a', 'weekend', 'trip', 'with', 'my',...
3       ['We', 'stayed', 'here', 'because', 'of', 'all...
4       ['Just', 'returned', 'from', 'a', 'week', 'in'...
5       ['We', 'stayed', 'at', 'the', 'Swissotel', 'fo...
6       ['I', 'just', 'left', 'the', 'Conrad', 'Chicag...
7       ['This', 'is', 'a', 'great', 'hotel', '!', 'Th...
8       ['Got', 'a', 'good', 'deal', 'on', 'a', 'weeke...
9       ['Thirty', 'years', 'ago', ',', 'we', 'had', '...
10      ['I', 'stayed', 'here', 'August', '20', 'throu...
11      ['The', 'Omni', 'Chicago', 'Hotel', 'I', 'am',...
12      ['excellent', 'staff', 'and', 'customer', 'ser...
13      ['I', 'am', 'a', 'fan', 'of', 'Kimpton', 'Hote...
14      ['I', 'stay', 'at', 'this', 'hotel', '2', 'tim...
15      ['The', 'InterContinental', 'Chicago', 'is', '...
16      ['This', 'hotel', 'was', 'wonderful', '!', 'I'...
17      ['I', 

In [19]:
#data['species_pos'] = data['species_pos'].astype(str).apply(lambda x: ', '.join(data['species_pos'].astype(str)))

In [20]:
data

,review,target,review_tokenized,species_tokens,species_lemma,species_pos
0,The Hard Rock Hotel Chicago has become my favo...,2,"(The, Hard, Rock, Hotel, Chicago, has, become,...","['The', 'Hard', 'Rock', 'Hotel', 'Chicago', 'h...","[the, hard, rock, hotel, chicago, have, become...","[DET, PROPN, PROPN, PROPN, PROPN, VERB, VERB, ..."
1,My wife and I had to stay downtown for event t...,2,"(My, wife, and, I, had, to, stay, downtown, fo...","['My', 'wife', 'and', 'I', 'had', 'to', 'stay'...","[-PRON-, wife, and, -PRON-, have, to, stay, do...","[ADJ, NOUN, CCONJ, PRON, VERB, PART, VERB, NOU..."
2,took a weekend trip with my wife. got a great ...,2,"(took, a, weekend, trip, with, my, wife, ., go...","['took', 'a', 'weekend', 'trip', 'with', 'my',...","[take, a, weekend, trip, with, -PRON-, wife, ....","[VERB, DET, NOUN, NOUN, ADP, ADJ, NOUN, PUNCT,..."
3,We stayed here because of all the great review...,2,"(We, stayed, here, because, of, all, the, grea...","['We', 'stayed', 'here', 'because', 'of', 'all...","[-PRON-, stay, here, because, of, all, the, gr...","[PRON, VERB, ADV, ADP, ADP, ADJ, DET, ADJ, NOU..."
4,Just returned from a week in Chicago with the ...,2,"(Just, returned, from, a, week, in, Chicago, w...","['Just', 'returned', 'from', 'a', 'week', 'in'...","[just, return, from, a, week, in, chicago, wit...","[ADV, VERB, ADP, DET, NOUN, ADP, PROPN, ADP, D..."
5,We stayed at the Swissotel for a special birth...,2,"(We, stayed, at, the, Swissotel, for, a, speci...","['We', 'stayed', 'at', 'the', 'Swissotel', 'fo...","[-PRON-, stay, at, the, swissotel, for, a, spe...","[PRON, VERB, ADP, DET, PROPN, ADP, DET, ADJ, N..."
6,I just left the Conrad Chicago and have nothin...,2,"(I, just, left, the, Conrad, Chicago, and, hav...","['I', 'just', 'left', 'the', 'Conrad', 'Chicag...","[-PRON-, just, leave, the, conrad, chicago, an...","[PRON, ADV, VERB, DET, PROPN, PROPN, CCONJ, VE..."
7,"This is a great hotel! The views are fabulous,...",2,"(This, is, a, great, hotel, !, The, views, are...","['This', 'is', 'a', 'great', 'hotel', '!', 'Th...","[this, be, a, great, hotel, !, the, view, be, ...","[DET, VERB, DET, ADJ, NOUN, PUNCT, DET, NOUN, ..."
8,Got a good deal on a weekend pacage in this Ho...,2,"(Got, a, good, deal, on, a, weekend, pacage, i...","['Got', 'a', 'good', 'deal', 'on', 'a', 'weeke...","[get, a, good, deal, on, a, weekend, pacage, i...","[VERB, DET, ADJ, NOUN, ADP, DET, NOUN, NOUN, A..."
9,"Thirty years ago, we had a tiny ""room"" and ind...",2,"(Thirty, years, ago, ,, we, had, a, tiny, "", r...","['Thirty', 'years', 'ago', ',', 'we', 'had', '...","[thirty, year, ago, ,, -PRON-, have, a, tiny, ...","[NUM, NOUN, ADV, PUNCT, PRON, VERB, DET, ADJ, ..."


In [21]:
#data['new_col'] = data[['species_tokens', 'species_pos']].apply(lambda x:'_'.join(x), axis=1)

In [22]:
#data["new_col1"] = data["species_tokens"].map(str) + data["species_pos"].map(str)

In [36]:
for i in range(0,1610):
    for j in range(0,len(df.iloc[i]['species_tokens'])):
        df.iloc[i]['new_col']=df.iloc[i]['species_tokens'][j]+df.iloc[i]['species_pos'][j]
        print(j)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/series.py:915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: list index out of range